In [10]:
from right_start_tools import main as rst


def generate_terraform_module_account_baseline(
        account_name: str,
        source: str,
) -> str:
    account_name_snake = account_name.lower().replace("-", "_")

    return (
        f'module "{account_name_snake}_account_baseline"'
        ' {\n'
        f'  source = "{source}"\n\n'
        f'  config = local.accounts_config["{account_name}"]\n'
        '}\n'
        )


In [9]:
def get_aws_account_id(session):
    # Assuming AWS credentials are configured in the environment or via config files
    sts_client = session.client('sts')
    account_id = sts_client.get_caller_identity()['Account']
    return account_id

In [2]:
from boto3.session import Session
# import main as rst

region = 'us-east-1'
profile_name = 'cristal-management'
# aws sso login --profile cristal-management
session = Session(profile_name=profile_name, region_name=region)

In [21]:
# module.athena_account_baseline_primary.aws_athena_workgroup.primary 
# module.athena_account_baseline_secondary.aws_athena_workgroup.primary
def gen_import_block(to: str, id: str) -> str:
    return f'import {{\n  to = {to}\n  id = "{id}"\n}}'

def gen_athena_workgroup_import_block(
        account_name: str,
        workgroup_name: str,
    ) -> str:
    account_name_snake = account_name.lower().replace("-", "_")
    
    return (
        gen_import_block(f'module.{account_name_snake}_account_baseline.module.athena_account_baseline_primary.aws_athena_workgroup.primary[0]', workgroup_name)
        + '\n\n' +
        gen_import_block(f'module.{account_name_snake}_account_baseline.module.athena_account_baseline_secondary.aws_athena_workgroup.primary[0]', workgroup_name)
    )


import {
  to = module.management_account_baseline.module.athena_account_baseline_primary.aws_athena_workgroup.primary[0]
  id = "primary"
}

import {
  to = module.management_account_baseline.module.athena_account_baseline_secondary.aws_athena_workgroup.primary[0]
  id = "primary"
}


In [24]:

from right_start_tools.tools import Tools
from right_start_tools.constants import CT_EXECUTION_ROLE_NAME

def gen_all_athena_workgroup_primary_import_block(
        accounts: list[rst.Account],
    ) -> str:
    import_blocks = []
    for account in accounts:
        import_blocks.append(
            gen_athena_workgroup_import_block(account.name, 'primary')
        )
    return '\n\n'.join(import_blocks)

def gen_all_terraform_module_account_baseline(
        accounts: list[rst.Account],
        source: str,
) -> str:
    module_blocks = []
    for account in accounts:
        module_blocks.append(
            generate_terraform_module_account_baseline(
                account_name=account.name,
                source=source,
            )
        )
    return '\n\n'.join(module_blocks)

t = Tools(session)

root_id = t.org.get_root_id()
structure = t.org.get_org_structure(root_id)
accounts = structure.all_accounts()


In [26]:
exclude_accounts = ["security-encryption"]

accs = [a for a in accounts if a.name not in exclude_accounts]

# print(generate_terraform_module_account_baseline(
#     account_name="management",
#     source="../../../../modules/external/account_baseline",
#     )
# )
# print(gen_athena_workgroup_import_block('management', 'primary'))

    
# gen_all_terraform_module_account_baseline(accs, "../../../../modules/external/account_baseline")


all_athena_workgroup_primary_import_blocks = gen_all_athena_workgroup_primary_import_block(accs)
print(all_athena_workgroup_primary_import_blocks)


import {
  to = module.security_log_archive_account_baseline.module.athena_account_baseline_primary.aws_athena_workgroup.primary[0]
  id = "primary"
}

import {
  to = module.security_log_archive_account_baseline.module.athena_account_baseline_secondary.aws_athena_workgroup.primary[0]
  id = "primary"
}

import {
  to = module.security_tooling_account_baseline.module.athena_account_baseline_primary.aws_athena_workgroup.primary[0]
  id = "primary"
}

import {
  to = module.security_tooling_account_baseline.module.athena_account_baseline_secondary.aws_athena_workgroup.primary[0]
  id = "primary"
}

import {
  to = module.production_workloads_1_account_baseline.module.athena_account_baseline_primary.aws_athena_workgroup.primary[0]
  id = "primary"
}

import {
  to = module.production_workloads_1_account_baseline.module.athena_account_baseline_secondary.aws_athena_workgroup.primary[0]
  id = "primary"
}

import {
  to = module.production_tools_backup_account_baseline.module.athena_account_